# Using Club ELO's API to gather ELO ratings of all clubs over each season

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
df1920 = pd.read_csv('../CSV/19-20.csv')
df1819 = pd.read_csv('../CSV/season-1819_csv.csv')

In [3]:
df1920.HomeTeam.unique()

array(['Liverpool', 'West Ham', 'Bournemouth', 'Burnley',
       'Crystal Palace', 'Watford', 'Tottenham', 'Leicester', 'Newcastle',
       'Man United', 'Arsenal', 'Aston Villa', 'Brighton', 'Everton',
       'Norwich', 'Southampton', 'Man City', 'Sheffield United',
       'Chelsea', 'Wolves'], dtype=object)

In [4]:
df1819.HomeTeam.unique()

array(['Man United', 'Bournemouth', 'Fulham', 'Huddersfield', 'Newcastle',
       'Watford', 'Wolves', 'Arsenal', 'Liverpool', 'Southampton',
       'Cardiff', 'Chelsea', 'Everton', 'Leicester', 'Tottenham',
       'West Ham', 'Brighton', 'Burnley', 'Man City', 'Crystal Palace'],
      dtype=object)

In [5]:
def Remove(duplicate): 
    """ a function to remove duplicates from a list """
    final_list = [] 
    for name in duplicate: 
        if name not in final_list: 
            final_list.append(name) 
    return final_list 

In [6]:
all_clubs = list(df1920.HomeTeam.unique()) + list(df1819.HomeTeam.unique())

In [7]:
club_list = Remove(all_clubs)
len(club_list)

23

In [9]:
clubs = club_list

new_clubs = []
for i in clubs:
    new_clubs.append(i.replace(" ", ""))  #remove any spaces as these don't work in the url for the API
    
clubs = new_clubs

In [10]:
# Commented out so don't accidently run the scraper unnecesarily
# You need to run this if doing from scratch though

#for i in clubs:
    # csv = requests.get('http://api.clubelo.com/{}'.format(i))
    
   #  with open('{}.csv'.format(i), 'w+') as f:
     #   f.write(csv.text)


#the csv files for each club were later moved to the CSV file however for the purpose of this notebook they were kept in the working directory


In [11]:
 #create an empty df to append the cleaned csv's to

ELO = pd.DataFrame()


for i in clubs:
    
    club_df = pd.read_csv('{}.csv'.format(i)) # open each csv
    
    # convert date column to datetime format
    try:
        club_df['Date'] = pd.to_datetime(club_df['To'])
    except:
        club_df['Date'] = pd.to_datetime(club_df['To'])
    # set date as index 
    club_df.set_index('Date', inplace=True, drop=True)
    # drop redundent columns
    club_df.drop(columns=['From', 'To', 'Level', 'Country', 'Rank'], inplace=True)
    
    # subset the relevant periods
    club_df = club_df[(club_df.index >= '2018-07') & (club_df.index <= pd.to_datetime('2019-12-30'))]
    
    club_df = club_df.resample('1D').ffill()
    
    # combine cleaned df with all_elo
    ELO = pd.concat([ELO, club_df])

In [12]:
ELO.shape

(11841, 2)

In [13]:
ELO.tail()

,Club,Elo
Date,,
2019-12-14,Cardiff,1571.067993
2019-12-15,Cardiff,1571.067993
2019-12-16,Cardiff,1571.067993
2019-12-17,Cardiff,1571.067993
2019-12-18,Cardiff,1571.067993


In [14]:
ELO.Club.sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
       'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield', 'Leicester', 'Liverpool', 'Man City', 'Man United',
       'Newcastle', 'Norwich', 'Sheffield United', 'Southampton',
       'Tottenham', 'Watford', 'West Ham', 'Wolves'], dtype=object)

In [15]:
standings = pd.read_csv('../CSV/LP.csv')

In [16]:
standings.Team.sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
       'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield', 'Leicester', 'Liverpool', 'Man City', 'Man United',
       'Newcastle', 'Norwich', 'Sheffield United', 'Southampton',
       'Tottenham', 'Watford', 'West Ham', 'Wolves'], dtype=object)

In [17]:
standings.Team.sort_values().unique() == ELO.Club.sort_values().unique() 
#double check all the team names are the same

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [18]:
standings.head()

,Position,Team,Points,Goal_Difference,Played,Date
0,1,Bournemouth,0,0,0,2018-08-10
1,1,Arsenal,0,0,0,2018-08-10
2,1,Brighton,0,0,0,2018-08-10
3,1,Burnley,0,0,0,2018-08-10
4,1,Cardiff,0,0,0,2018-08-10


In [19]:
standings.set_index('Date', drop=False, inplace = True)

In [20]:
standings['Link'] = [str(i)+n[:5] for i, n in zip(standings.index, standings.Team)] 
#create specific link between club and data to act as a key for merging

In [22]:
ELO['Link'] = [str(i)[:10]+n[:5] for i, n in zip(ELO.index, ELO.Club)]

In [23]:
ELO.head(20)

,Club,Elo,Link
Date,,,
2018-07-26,Liverpool,1913.856445,2018-07-26Liver
2018-07-27,Liverpool,1913.856445,2018-07-27Liver
2018-07-28,Liverpool,1913.856445,2018-07-28Liver
2018-07-29,Liverpool,1913.856445,2018-07-29Liver
2018-07-30,Liverpool,1913.856445,2018-07-30Liver
2018-07-31,Liverpool,1913.856445,2018-07-31Liver
2018-08-01,Liverpool,1913.856445,2018-08-01Liver
2018-08-02,Liverpool,1913.885986,2018-08-02Liver
2018-08-03,Liverpool,1913.885986,2018-08-03Liver


In [24]:
final = pd.merge(left = ELO, right = standings, left_on = 'Link', right_on = 'Link', how = 'left')

In [25]:
final.isnull().sum()

Club                  0
Elo                   0
Link                  0
Position           3349
Team               3349
Points             3349
Goal_Difference    3349
Played             3349
Date               3349
dtype: int64

In [26]:
final.shape

(11841, 9)

In [27]:
df = final.ffill()

In [28]:
df.isnull().sum().sum()

90

In [29]:
df.isnull().sum()

Club                0
Elo                 0
Link                0
Position           15
Team               15
Points             15
Goal_Difference    15
Played             15
Date               15
dtype: int64

In [30]:
df.dropna(axis = 0, inplace = True)

In [31]:
LP_ELO = df.to_csv('../CSV/LP_ELO.csv', index=False)

In [32]:
df.head(20) #just to have a look and see if all is in order

,Club,Elo,Link,Position,Team,Points,Goal_Difference,Played,Date
15,Liverpool,1914.838867,2018-08-10Liver,1.0,Liverpool,0.0,0.0,0.0,2018-08-10
16,Liverpool,1915.229004,2018-08-11Liver,1.0,Liverpool,0.0,0.0,0.0,2018-08-10
17,Liverpool,1915.229004,2018-08-12Liver,1.0,Liverpool,0.0,0.0,0.0,2018-08-10
18,Liverpool,1915.229004,2018-08-13Liver,1.0,Liverpool,3.0,4.0,1.0,2018-08-13
19,Liverpool,1915.229004,2018-08-14Liver,1.0,Liverpool,3.0,4.0,1.0,2018-08-14
20,Liverpool,1915.229004,2018-08-15Liver,1.0,Liverpool,3.0,4.0,1.0,2018-08-15
21,Liverpool,1918.966187,2018-08-16Liver,1.0,Liverpool,3.0,4.0,1.0,2018-08-16
22,Liverpool,1918.966187,2018-08-17Liver,1.0,Liverpool,3.0,4.0,1.0,2018-08-17
23,Liverpool,1918.966187,2018-08-18Liver,1.0,Liverpool,3.0,4.0,1.0,2018-08-18
24,Liverpool,1918.966187,2018-08-19Liver,5.0,Liverpool,3.0,4.0,1.0,2018-08-19
